In [ ]:
import json
import pandas as pd
from text2graph.llm import ask_llm, OpenSourceModel
from text2graph import lemmatizer

In [ ]:
print(f"Supported open-source (OSS) models: {[m.value for m in OpenSourceModel]}")

- `mixtral`: top open-source llm model based on [chatbot arean](https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard), ELO: 1118 ~= GPT3.5 turbo
- `openhermes`: decent open-source small-ish (7b) model, ELO: 1078 (won't test on this yet, Bill is using this with somewhat decent results in similar use case?)

## Extract locations based-on Shanan's example

In [ ]:
NEGATIVE_EXAMPLE = """
This community-vetted CO2 syn- thesis represents the most reliable data avail- able to date and a means to improve our understanding of past changes in global cli- mate and carbon cycling as well as organismal evolution. However, this effort is still incomplete. Data remain sparse during the earlier part of the record and in some instances are domi- nated by estimates from a single proxy system. Generating a paleo-CO2 record with even greater confidence will require further research using multiple proxies to fill in data gaps and increase overall data resolution, resolve discrepancies between estimates from contemporaneous proxy analyses, reduce uncertainty o
"""

POSITIVE_EXAMPLE = """
The top of the Sauk megasequence in Minnesota is at the unconformable contact of the Shakopee Formation with the St. Peter Sandstone. Younger rocks are present beneath the St. Peter Sandstone on the southern and east- ern flanks of the Ozark dome, where the upper Sauk succession includes the Roubidoux, Jefferson City, Cotter, Powell – Smithville – Black Rock, and Everton units in that stratigraphic order (Ethington et al., 2012; Palmer et al., 2012). The Shakopee Formation is equivalent to some lower part of this succession, but sparse inverte- brate faunas and long-ranging conodonts in these units preclude correlation with high resolution. The Jasper Member of the Everton Formation of northern Arkansas contains conodonts of the Histiodella holodentata Biozone, which demonstrates the latest early Whiterockian age for the top of the rocks of the GACB in that region. No faunal evidence is available there for the age of the base of the St. Peter Sandstone. The boundary between the Sauk and Tippecanoe megasequences may be a cor- relative conformity in the Reelfoot rift of southeastern Missouri and northeastern Missouri, but this has not been demonstrated.
"""

In [Shanan's example](https://chat.openai.com/share/d14a5163-ddb4-4f28-b6d4-bbf4839f62cb), he forced the model to perform a guided 2 steps chain-of-thought.

1. What locations are mentioned in the following paragraph?
2. What geological entities are associated with those locations?

We can try to make an exact replication on our OSS models.  

In [ ]:
q1 = "What locations are mentioned in the following paragraph?"
q2 = "What geological entities are associated with those locations?"

message = {"role": "user", "content": f"{q1}\n{NEGATIVE_EXAMPLE}"}
response = ask_llm(messages=[message], model="mixtral")
print(response)

For reference, this is the exact return from Shanan's example:
```
The paragraph does not mention specific locations. Instead, it discusses data synthesis and the need for further research to improve our understanding of past changes in global climate and carbon cycling. Therefore, specific locations are not mentioned in this paragraph.
``````

Somewhat close enough, perhaps GPT prefers a bit simpler wordings.

In [ ]:
message = {"role": "user", "content": f"{q1}\n{POSITIVE_EXAMPLE}"}
response = ask_llm(messages=[message], model="mixtral")
print(response)

For reference, this is the exact return from Shanan's example:
```
The locations mentioned in the paragraph are:

Minnesota
Ozark dome (southern and eastern flanks)
Northern Arkansas
Reelfoot rift (southeastern Missouri and northeastern Missouri)
```

Simulate the conversation with follow-up question:

In [ ]:
messages = [message]  # Last message
messages.append({"role": "assistant", "content": response})  # Last response
messages.append({"role": "user", "content": q2})
response = ask_llm(messages=messages, model="mixtral")
print(response)

For reference, this is the exact return from Shanan's example:
```
The geological entities associated with the mentioned locations are:

Minnesota: Sauk megasequence, Shakopee Formation, St. Peter Sandstone.
Ozark dome (southern and eastern flanks): Sauk megasequence, Roubidoux Formation, Jefferson City Formation, Cotter Formation, Powell-Smithville-Black Rock Formation, Everton Formation.
Northern Arkansas: Everton Formation, Jasper Member.
Reelfoot rift (southeastern Missouri and northeastern Missouri): Sauk megasequence, Tippecanoe megasequence.
``````

Interim conclusion:

In this particular use case, perhaps `mixtral` is suffice?

More issues to consider:

1. Can we speed it up a bit by Reducing query round trip from 2 to 1?
1. Can we return json format instead of plain text?
1. How to lemmatize location or geo-entities? (Try traditional methods first, then we can try embedding distance based method)
1. Test on larger test set. Use a subset of Devesh's test set. Perhaps 30 examples first, scale up later if we reach a good result. Need manual labeling. (Devesh's test set don't have labels) 
1. Serve API endpoint for user testing.
1. Serve Demo for user testing.

Backlog low-priority items:

1. Geo-coding for extracted locations.
1. How to use the Stratigraphy and Lithology list from MacroStrat?
1. Multi-agent setup to improve quality.
1. Logging and monitoring: Collect user feedback for fine-tuning (e.g., prompt-tune (interactive or generative) or Direct Preference Optimization (need around 20k labeled data))


## Speed up experiment: Zero-shot CoT

- Borrowing some system prompt idea from Sky's [ta2-extraction example](https://github.com/DARPA-CRITICALMAAS/ta2-extraction/blob/master/prompts.py)
- Try to reduce round-trip from 2 to 1.
- Improve response format from free text to json-like.


In [ ]:
def get_prompt(text: str) -> list[dict]:
    """V0 geo-location prompting."""

    system_prompt = {
        "role": "system",
        "content": "You are a geology expert and you are very good in understanding mining reports. Think step by step: What locations are mentioned in the following paragraph? and What geological entities are associated with those locations? Return in json format like this: {'location1': ['entity1', 'entity2', ...], 'location2': ['entity3', 'entity4', ...]}. Return an empty dictionary if there is no location.",
    }
    user_prompt = {"role": "user", "content": text}
    return [system_prompt, user_prompt]


def experiment1(text: str) -> str:
    """V0 geo-location experiment."""
    response = ask_llm(messages=get_prompt(text), model="mixtral")
    return response

In [ ]:
print(experiment1(NEGATIVE_EXAMPLE))

In [ ]:
print(experiment1(POSITIVE_EXAMPLE))

For reference, this is the exact return from Shanan's example:
```
The geological entities associated with the mentioned locations are:

Minnesota: Sauk megasequence, Shakopee Formation, St. Peter Sandstone.
Ozark dome (southern and eastern flanks): Sauk megasequence, Roubidoux Formation, Jefferson City Formation, Cotter Formation, Powell-Smithville-Black Rock Formation, Everton Formation.
Northern Arkansas: Everton Formation, Jasper Member.
Reelfoot rift (southeastern Missouri and northeastern Missouri): Sauk megasequence, Tippecanoe megasequence.
``````

Quality check talking points. 

- should `younger rocks` counts as geo-entity? If yes -> `mixtral` wins
- `Histiodella holodentata Biozone` is a geo-entity? If yes -> `mixtral` wins


## Micro testset (n=30)

In [ ]:
# Run once
# critical_maas_knowledge_graph_testset = pd.read_parquet("data/formation_sample.parquet.gzip")
# testset_micro = critical_maas_knowledge_graph_testset.sample(30)
# testset_micro.to_parquet("data/testset_micro.parquet.gzip", compression="gzip")

testset_micro = pd.read_parquet("data/testset_micro.parquet.gzip")
testset_micro.sample(3)

In [ ]:
results = {
    "note": "V0 geo-location experiment.",
    "testset_path": "data/testset_micro.parquet.gzip",
    "input": [],
    "output": [],
}

for i, row in testset_micro.iterrows():
    input = row["paragraph"]
    output = experiment1(input)
    results["input"].append(input)
    results["output"].append(output)

In [ ]:
with open("results.json", "w") as f:
    json.dump(results, f, indent=4)

In [ ]:
def print_output(index: int, results: dict) -> None:
    print(f"Input: {results['input'][index]}")
    print()
    print(f"Output: {results['output'][index]}")


for i in range(30):
    print_output(i, results)
    print("=" * 200)

## Lemmatization experiment (Somewhat not working yet, can skip for now)

Problem statement: The current geographic details and entities are overly specific for generating an aggregated graph. We must simplify them into broader terms, which will then serve as the basis for the nodes in the knowledge graph.

tl;dr;

- traditional methods don't work, they are more design for word level lemmatization, not phrase level.

Options:

1. nltk
1. spacy
1. embedding distance
1. llm

### nltk lemmatization

In [ ]:
test_locations = set()
for r in results["output"]:
    try:
        test_locations.update(json.loads(r).keys())
    except json.JSONDecodeError:
        print(r)
        pass

Note to self: TODO: need better json format.

- "," in location
- other special characters in location

In [ ]:
test_locations = list(test_locations)
print(test_locations)

In [ ]:
nltk = lemmatizer.NLTK()
spacy = lemmatizer.Spacy()
llm = lemmatizer.LLM()

In [ ]:
df_lemma = pd.DataFrame(
    {
        "locations": test_locations,
        "nltk": [nltk.lemmatize(location) for location in test_locations],
        "spacy": [spacy.lemmatize(location) for location in test_locations],
        "llm": [llm.lemmatize(location) for location in test_locations],
    }
)

In [ ]:
df_lemma

Should have much room for improvement...